# Multivariate analysis between clustering

This experiment notebook runs multivariate analysis and ANOVA tests between features between the clusters. More specifically:
* We run multivariate ANOVA tests between the obtained clusters.
* We run analysis of covariance between the clusters (THIS is ANOVA, right?)
* We create random clustering and run the same tests there to make sure that the clustering gives reasonable results.

**Imports**

In [2]:
import numpy as np
import pandas as pd
import sys
import simlr_ad
from utils.data_utils import load_all_data
from utils.utils import compute_simlr, feat_ranking

**Configuration variables**

In [3]:
# Paths
existing_cluster = True                                # Compute the clustering again or use an existing one
cluster_path = "results/extendeddata_cluster/cluster_data.csv"   # Path of the existing cluster, if applicable
covariate_path = "data/useddata_homo_abeta_plasma_meta.csv"                 # Path of the covariance data frame (.csv)
feature_path = "data/UCSDVOL.csv"                    # Path of the feature path (.csv)

# Parameters of the cluster creation
clusters = 3                                          # Number of cluster for clustering computation

# Testing parameters
rd_seed = 1714                                          # Random seed for experiment replication

# Data description
* *covariate_data* contains the features in the covariate space (blood, etc).
* *feature_data* contains the feature in the covariate space (volumes/atrophy).

In [4]:
covariate_data, cov_names, feature_data, feature_names = load_all_data(covariate_path, feature_path)

**Create or load base clustering***

In [5]:
if existing_cluster:
    # Load existent
    c_data = pd.read_csv(cluster_path)
else:
    # Compute base clustering
    y_b, S, F, ydata, alpha = compute_simlr(
        np.array(covariate_data_new[cov_names]), clusters)


**Create other clusterings, to compare later**

In [6]:
# Random clustering
rd_c = np.random.randint(1,clusters+1, size=len(c_data))

# kmeans clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=clusters, random_state = rd_seed).fit(covariate_data[cov_names])
km_c = km.labels_ +1

In [7]:
# Check distribution of DX in each cluster
unique, counts = np.unique(covariate_data.DX_bl.values, return_counts=True)
print(dict(zip(unique, counts)))

for c in range(1, clusters+1):
    c1 = covariate_data[c_data['C'].values == c].DX_bl.values
    unique, counts = np.unique(c1, return_counts=True)
    print(dict(zip(unique, counts)))

{'AD': 85, 'CN': 52, 'LMCI': 161}
{'AD': 20, 'CN': 24, 'LMCI': 68}
{'AD': 36, 'CN': 19, 'LMCI': 75}
{'AD': 29, 'CN': 9, 'LMCI': 18}


**Asumptions of normality and homogeneity variance **

In [8]:
# Which clustering to use?
# We could use k-means, we could use random, or another one
c_labels = c_data['C'].values

## Same calculations, but between clusters AND diagnostic groups
# Compute new labels
c_data["C_DX"] =  c_data[['C', 'DX']].apply(lambda x: '_'.join(x.map(str)), axis=1)
c_labels_dx = c_data["C_DX"].values

print(feature_names)

# Only between cluster groups
## Results: We cannot assume normality and variability in the 
for f in feature_names:
    print(f)
    
    curr_feat = feature_data[f]
    data_arr = []
    for c in set(c_labels):
        c1 = curr_feat[c_labels == c].values.tolist()
        normal_c1 = stats.shapiro(c1)
        print(normal_c1)
        # Variance assumptions
        data_arr.append(c1)
    
    var_c1 = stats.levene(data_arr[0],data_arr[1],data_arr[2])
    print(var_c1)

['VENTRICLES' 'LHIPPOC' 'RHIPPOC' 'LINFLATVEN' 'RINFLATVEN' 'LMIDTEMP'
 'RMIDTEMP' 'LINFTEMP' 'RINFTEMP' 'LFUSIFORM' 'RFUSIFORM' 'LENTORHIN'
 'RENTORHIN']
VENTRICLES


NameError: name 'stats' is not defined

In [9]:
# Between all cognitive and cluster groups
# Check for 1) Normality, 2) Variance Homoeneity in each of the possible combinations of values.
## Results: we cannot assume normality and variance homogeneity in all the groups.
from scipy import stats
for c in feature_names:
    print(c)
    curr_feat = feature_data[c]
    data_arr = []
    for l1 in set(c_labels_dx):
        print("Testing for: " + l1)
        c1 = curr_feat[c_labels_dx == l1].values.tolist()
        # Normality assumptions
        normal_c1 = stats.shapiro(c1)
        print(normal_c1)
        data_arr.append(c1)
    # Variance assumptions
    var_c1 = stats.levene(*data_arr)
    print(var_c1)


VENTRICLES
Testing for: 3.0_CN
(0.9172245264053345, 0.058114923536777496)
Testing for: 1.0_LMCI
(0.9433903694152832, 5.514952863450162e-05)
Testing for: 2.0_LMCI
(0.9626418948173523, 0.09603749215602875)
Testing for: 2.0_CN
(0.9083295464515686, 0.233001247048378)
Testing for: 3.0_AD
(0.8886845111846924, 0.0007815034477971494)
Testing for: 1.0_CN
(0.894244909286499, 0.03832496330142021)
Testing for: 3.0_LMCI
(0.9407792091369629, 1.2916758578285226e-06)
Testing for: 2.0_AD
(0.9486952424049377, 0.11221432685852051)
Testing for: 1.0_AD
(0.9817771315574646, 0.9092001914978027)
LeveneResult(statistic=1.4508365276536008, pvalue=0.1728322539005003)
LHIPPOC
Testing for: 3.0_CN
(0.9591532945632935, 0.4464833736419678)
Testing for: 1.0_LMCI
(0.9889426827430725, 0.4201415479183197)
Testing for: 2.0_LMCI
(0.9801239371299744, 0.5183107256889343)
Testing for: 2.0_CN
(0.9506474137306213, 0.6522375345230103)
Testing for: 3.0_AD
(0.967496395111084, 0.28492528200149536)
Testing for: 1.0_CN
(0.97239935398

### Various statistical tests

ONE WAY ANOVA, TUKEYHSD, MULTICOMPARISON

We assume normality, variance homogeneity, and independence between samples.

TODO: ADD check to those assumptions to verify if they are correct or not.

** Important: all these tests should be done on Freesurfer, using functio mriglmfit to check for stadistically significant differences**

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind


In [ ]:

# Between clusters and for each variable, not taking into account diagnostic labels
for c in feature_names:
    print(c)
    curr_feat = feature_data[c]
    mc1 = MultiComparison(feature_data[c], c_labels_dx)
    res1 = mc1.tukeyhsd()
    res2 = mc1.allpairtest(ttest_ind)
    # c1 = curr_feat[c_labels == 1].values.tolist()
    # c2 = curr_feat[c_labels == 2].values.tolist()
    # c3 = curr_feat[c_labels == 3].values.tolist()
    st,pvalue = stats.f_oneway(c1, c2, c3)
    print(res1.summary())
    print(res2[0].as_text())
    print('ANOVA: F-value: ' + str(st) + ' P-value: ' + str(pvalue))
    print('')


**Two-way anova, with DX and cluster label**
https://statistics.laerd.com/spss-tutorials/two-way-manova-using-spss-statistics.php


In [17]:
## Compute two way anova
# Prepare data
print(set(c_labels_dx))
import rpy2
%load_ext rpy2.ipython
#for c in feature_names:
c = "VENTRICLES"
curr_feat = feature_data[c]

c1_AD = curr_feat[c_labels_dx == "1.0_AD"].values
c1_MCI = curr_feat[c_labels_dx == "1.0_LMCI"].values
c1_CN = curr_feat[c_labels_dx == "1.0_CN"].values

c2_AD = curr_feat[c_labels_dx == "2.0_AD"].values
c2_MCI = curr_feat[c_labels_dx == "2.0_LMCI"].values
c2_CN = curr_feat[c_labels_dx == "2.0_CN"].values

c3_AD = curr_feat[c_labels_dx == "3.0_AD"].values
c3_MCI = curr_feat[c_labels_dx == "3.0_LMCI"].values
c3_CN = curr_feat[c_labels_dx == "3.0_CN"].values

%Rpush c1_AD c1_MCI c1_CN c2_AD c2_MCI c2_CN c3_AD c3_MCI c3_CN

%R Factor1 <- c('c1','c1','c1','c2','c2','c2','c3','c3','c3')
%R Factor2 <- c('AD','MCI','CN','AD','MCI','CN','AD','MCI','CN')
%R idata <- data.frame(Factor1, Factor2)

#make sure the vectors appear in the same order as they appear in the dataframe
%R Bind <- cbind(c1_AD, c1_MCI, c1_CN, c2_AD, c2_MCI, c2_CN, c3_AD, c3_MCI, c3_CN)
%R model <- lm(Bind~1)

%R library(car)
%R analysis <- Anova(model, idata=idata, idesign=~Factor1*Factor2, type="III")
%R -o anova_summary anova_summary = summary(analysis)
print(anova_summary)

{'3.0_CN', '1.0_LMCI', '2.0_LMCI', '2.0_CN', '3.0_AD', '1.0_CN', '3.0_LMCI', '2.0_AD', '1.0_AD'}
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython

Type III Repeated Measures MANOVA Tests:

------------------------------------------
 
Term: (Intercept) 

 Response transformation matrix:
       (Intercept)
c1_AD            1
c1_MCI           1
c1_CN            1
c2_AD            1
c2_MCI           1
c2_CN            1
c3_AD            1
c3_MCI           1
c3_CN            1

Sum of squares and products for the hypothesis:
            (Intercept)
(Intercept)    1315.556

Multivariate Tests: (Intercept)
                 Df test stat approx F num Df den Df     Pr(>F)    
Pillai            1  0.967297 5117.063      1    173 < 2.22e-16 ***
Wilks             1  0.032703 5117.063      1    173 < 2.22e-16 ***
Hotelling-Lawley  1 29.578401 5117.063      1    173 < 2.22e-16 ***
Roy               1 29.578401 5117.063      1    173 < 2.22e-16 ***
---
Signif

** Wilcoxon test and other non-parametric tests **
Wicoxon and whitney require n>20 samples in each group. This is not achieved in some of the smaller groups... In almost e